In [26]:
# 01. Import thư viện, load dữ liệu và chuẩn hóa tên cột

import os
import pandas as pd
import numpy as np

# Hàm chuẩn hóa tên cột: bỏ ký tự đặc biệt, khoảng trắng, đổi sang lowercase
def clean_col_name(c):
    c = c.strip()
    c = c.replace(" ", "_").replace(".", "_").replace("/", "_")
    c = c.replace("(", "").replace(")", "").replace("<", "lt_").replace(">", "gt_")
    while "__" in c:
        c = c.replace("__", "_")
    return c.lower().strip("_")

data_path = "data/raw/SharkIncident.csv"
df = pd.read_csv(data_path, low_memory=False)

# Số dòng và cột
print("- Số dòng:", df.shape[0])
print("- Số cột:", df.shape[1])

# Chuẩn hóa tên các cột
df.columns = [clean_col_name(c) for c in df.columns]

print("- 5 dòng đầu:")
display(df.head())


- Số dòng: 1283
- Số cột: 60
- 5 dòng đầu:


,uin,incident_month,incident_year,victim_injury,state,location,latitude,longitude,site_category,site_category_comment,...,spring_or_neap_tide,tidal_cycle,wind_condition,weather_condition,air_temperature_°c,personal_protective_device,deterrent_brand_and_type,data_source,reference,unnamed:_59
0,1,1,1791,fatal,NSW,sydney (near),-33.86,151.2000,coastal,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,book,"shark&survl, whitley 1958, book ref 1793",NaN
1,2,3,1803,injured,WA,"shark bay, faure island",-25.8826,113.9226,coastal,bay to open ocean,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,book,"balgridge,green,taylor,whitley 1940",NaN
2,3,1,1807,injured,NSW,"sydney harbour, cockle bay",-33.8661,151.2010,estuary/harbour,bay,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,media outlet,sydney gazette 18.1.1807,NaN
3,4,1,1820,fatal,TAS,"sweetwater point, pitt water",-42.8025,147.4868,estuary/harbour,bay to open ocean,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,witness account,"shark&survl, c. black researcher",NaN
4,5,1,1825,injured,NSW,"sydney harbour, kirribili point",-33.8527,151.2188,estuary/harbour,harbour,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,media outlet,maitland daily mercury 13.11.1899,NaN


In [27]:
# 02. Bỏ các cột thiếu dữ liệu và xóa các dòng lặp lại

# Drop cột thiếu > 99% dữ liệu vì các cột này không thể cung cấp insight hữu ích, chỉ tăng nhiễu và làm sai lệch thống kê.
missing_threshold = 0.99
missing_percent = df.isna().sum() / len(df) #missing percent
cols_drop = missing_percent[missing_percent > missing_threshold].index.tolist()
df = df.drop(columns=cols_drop)
print(f"- Xóa {len(cols_drop)} cột:")
print(cols_drop)

# Xử lý duplicated rows
dups = df.duplicated()
print(f"\n- Tổng số dòng trùng lặp: {dups.sum()}")
if dups.any():
    # Loại bỏ các bản ghi trùng lặp chính xác, mỗi dòng phải là một sự kiện duy nhất.
    df = df.drop_duplicates(keep='first')
    print(f"- Đã xóa cách dòng trùng lặp, shape sau khi xóa: {df.shape}")


# Missing values sau khi drop cột
missing_overview = df.isna().sum().sort_values(ascending=False).to_frame("missing_count")
missing_overview["missing_percent"] = (missing_overview["missing_count"] / len(df) * 100).round(2)
print("- Missing value sau xử lí:") 
display(missing_overview.head(15)) #In top 15


- Xóa 5 cột:
['fish_speared?', 'clothing_pattern', 'spring_or_neap_tide', 'deterrent_brand_and_type', 'unnamed:_59']

- Tổng số dòng trùng lặp: 0
- Missing value sau xử lí:


,missing_count,missing_percent
other_clothing_colour,1261,98.29
fin_colour,1251,97.51
tidal_cycle,1249,97.35
air_temperature_°c,1240,96.65
personal_protective_device,1227,95.64
weather_condition,1226,95.56
teeth_recovered,1223,95.32
diversionary_action_outcome,1217,94.86
water_visability_m,1211,94.39
wind_condition,1202,93.69


### Vì sao chọn loại bỏ các cột có >99% missing thay vì xóa dòng?

- Các cột thiếu quá 99% dữ liệu không còn đủ thông tin để phân tích hay huấn luyện mô hình. Giữ lại chỉ làm tăng nhiễu và độ phức tạp.

- Nếu cố gắng giữ lại các cột này, ta gần như phải điền toàn bộ giá trị, khiến biến trở nên kém ý nghĩa và dễ gây sai lệch.

- Việc drop cột giúp giữ lại tối đa số dòng, nếu drop dòng thay vì drop cột thì sẽ làm mất dữ liệu quan sát.


In [28]:
# 03. Xử lý missing dữ liệu số và chữ

# A. Các cột số
num_cols = df.select_dtypes(include=['number']).columns.tolist() # Lấy các cột số
# Xử lý giá trị vô lý (âm hoặc 0 sai) 
placeholder_keywords = ["length", "age", "distance", "total_water_depth", "visability", "people"]
placeholder_cols = [
    c for c in num_cols 
    if any(k in c.lower() for k in placeholder_keywords)
]

for c in placeholder_cols:
    # số âm 
    mask_negative = df[c] < 0
    df.loc[mask_negative, c] = np.nan
    
    # số 0, ngoại trừ depth (vì ở đây ta xét 0 = ngay mặt nước)
    if c.lower() not in ["depth.of.incident.m"]:
        mask_zero = df[c] == 0
        df.loc[mask_zero, c] = np.nan

# B. Thay missing của cột số thành -1
for c in num_cols:
    df[c] = df[c].fillna(-1)

# C. Thay missing của cột categories thành 'Unknown'
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
for c in cat_cols:
    df[c] = df[c].fillna("Unknown")



### Vì sao chọn cách thay giá trị này?

- Chọn thay missing value và các giá trị sai (âm hoặc bằng 0) của các cột số thành –1 và thay missing value của các cột phân loại thành “Unknown” vì đây là dữ liệu ghi nhận thực tế. Nếu điền median hoặc mode sẽ tạo ra giá trị không có cơ sở và làm sai lệch dữ liệu gốc.

- Riêng cột depth_of_incident_m vẫn giữ giá trị 0 vì theo ngữ cảnh dữ liệu, 0 được xem là sự cố xảy ra ngay trên mặt nước hoặc sát bờ biển.

In [29]:
# 04. Thống kê outlier 

rows = []  # danh sách các dòng để tạo bảng
for col in num_cols:
    series = df[col].dropna()
    if len(series) == 0:
        continue  # bỏ cột toàn NaN
    # Tính Q1, Q3 và IQR
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = series[(series < lower) | (series > upper)]
    # Thêm vào bảng
    rows.append({
        "Cột": col,
        "Số lượng outlier": len(outliers),
        "Tỷ lệ (%)": round(len(outliers) / len(series) * 100, 3),
        "Giá trị nhỏ nhất": series.min(),
        "Giá trị lớn nhất": series.max(),
        "Ngưỡng dưới": lower,
        "Ngưỡng trên": upper
    })

# Tạo DataFrame kết quả
outlier_df = pd.DataFrame(rows)

print("Bảng thống outlier\n")
outlier_df


Bảng thống outlier



,Cột,Số lượng outlier,Tỷ lệ (%),Giá trị nhỏ nhất,Giá trị lớn nhất,Ngưỡng dưới,Ngưỡng trên
0,uin,0,0.000,1.0000,1285.0,-640.0000,1924.0000
1,incident_month,0,0.000,1.0000,12.0,-10.0000,22.0000
2,incident_year,3,0.234,1791.0000,2025.0,1815.5000,2131.5000
3,longitude,205,15.978,96.8192,4034.0,117.9936,172.7012
4,no_sharks,114,8.885,-1.0000,10.0,1.0000,1.0000
5,people_lt_3m,93,7.249,-1.0000,12.0,-1.0000,-1.0000
6,people_3-15m,84,6.547,-1.0000,40.0,-1.0000,-1.0000
7,water_temperature_°c,93,7.249,-1.0000,29.0,-1.0000,-1.0000
8,water_visability_m,55,4.287,-1.0000,100.0,-1.0000,-1.0000
9,distance_to_shore_m,222,17.303,-1.0000,280000.0,-32.5000,51.5000


### Nhận xét Outlier

- uin, incident_month không có outlier → dữ liệu ổn.

- incident_year có vài outlier do năm ghi lệch nhưng không đáng kể.

- longitude có nhiều outlier có thể do lỗi nhập liệu.

- Các cột no_sharks, people*, water* có outlier chủ yếu vì chứa -1 nên không phải bất thường.

- distance_to_shore_m có outlier lớn.

### Không xử lý outlier

- Quyết định không xử lý outlier vì đây là dữ liệu ghi nhận thực tế. Việc thay đổi hay cắt giá trị outlier có thể làm mất thông tin thật và làm giảm độ chính xác khi phân tích.

- Thay vào đó, chỉ thống kê outlier bằng IQR để tham khảo và đánh giá chất lượng dữ liệu, không can thiệp vào giá trị gốc.

In [ ]:
# 05. Lưu dữ liệu đã làm sạch

out_dir = "data/processed"
os.makedirs(out_dir, exist_ok=True)
out_path = os.path.join(out_dir, "SharkIncident_cleaned.csv")
df.to_csv(out_path, index=False)
print(f"\nĐã lưu tập dữ liệu tại: {out_path}")


Dữ liệu không còn missing value.

Đã lưu tập dữ liệu tại: data/processed\SharkIncident_cleaned.csv
